In [1]:
import numpy as np
import torch
import datasets
import pandas as pd
# import editdistance
import transformers
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from accelerate import Accelerator
from sklearn.model_selection import StratifiedShuffleSplit
import os
from tqdm import tqdm
import re

# model_dir = '/home/user/ksharma/ks_thesis/saved_models/automated_model_saves/t5-large_ep-15_ga-1_b-4_lr-0.0003_wd-0.01/checkpoint-627'
# model_pretrained = AutoModelForSeq2SeqLM.from_pretrained(model_dir)
# tokenizer_pretrained = AutoTokenizer.from_pretrained(model_dir)

# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# # Create a new column 'generated_output' to store the generated output text
# results_FLAN_T5_XXL['generated_text'] = ''
# results_FLAN_T5_XXL['generated_skills_all'] = ''

# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model_pretrained.to(device)

# # Tokenize the input text and add the generated output text to the DataFrame
# for index, row in tqdm(results_FLAN_T5_XXL.iterrows(), total=len(results_FLAN_T5_XXL), desc="Generating Text"):
# # for index, row in test_set[:2].iterrows():
#     input_text = row['input']
#     input_ids = tokenizer_pretrained.encode(input_text, return_tensors="pt")
#     input_ids = input_ids.to(device)

#     # Generate the output text
#     output_ids = model_pretrained.generate(input_ids, max_new_tokens=280)
#     output_text = tokenizer_pretrained.decode(output_ids[0], skip_special_tokens=True)

#     # Assign the generated output text to the corresponding row in the DataFrame
#     results_FLAN_T5_XXL.at[index, 'generated_text'] = output_text

#     gen_skills = re.findall(r'(?:SoftSkill>|SoftSkill_C>)(.*?)(?:/SoftSkill>|/SoftSkill_C>)', output_text)
#     results_FLAN_T5_XXL.at[index, 'generated_skills_all'] = gen_skills



/home/user/ksharma/miniconda3/envs/ks_tars_thesis/lib/python3.9/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.0.3) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/user/ksharma/miniconda3/envs/ks_tars_thesis did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/lib64')}
  warn(msg)
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: No libcudart.so found! Install CUDA or the cudatoolkit package (anaconda)!
  warn(msg)


In [2]:
import pandas as pd
results_FLAN_T5_XXL = pd.read_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/FLAN_T5_XXL/model_generation.csv")
# results_FLAN_T5_XXL['expected_skills'] = results_FLAN_T5_XXL['expected_text'].str.findall(r'<SoftSkill>(.*?)</SoftSkill>')
results_FLAN_T5_XXL = results_FLAN_T5_XXL.drop(columns='Unnamed: 0', axis = 0)
results_FLAN_T5_XXL['expected_skills_all'] = results_FLAN_T5_XXL['expected_text'].str.findall(r'<SoftSkill>(.*?)<\/SoftSkill')
results_FLAN_T5_XXL['generated_skills_all'] = results_FLAN_T5_XXL['generated_text'].str.findall(r'(?:SoftSkill>|SoftSkill_C>)(.*?)(?:/SoftSkill>|/SoftSkill_C>)')
results_FLAN_T5_XXL.head(4)

,orig_idx,input,expected_text,Problem type,expected_skills,valid,generated_text,generated_skills,expected_skills_all,generated_skills_all
0,752,Wir wenden uns an einen gelernten Uhrmacher m...,Wir wenden uns an einen gelernten Uhrmacher m...,MC,"['dienstleitungsorientierten handeln', 'verant...",1,Wir wenden uns an einen gelernten Uhrmacher mi...,"['dienstleitungsorientierten handeln', 'verant...","[dienstleitungsorientierten handeln, verantwor...","[dienstleitungsorientierten handeln, verantwor..."
1,765,Ihre <SoftSkill>Arbeitsweise ist proaktiv</Sof...,Ihre <SoftSkill>Arbeitsweise ist proaktiv</Sof...,HOWA,"['Arbeitsweise ist proaktiv', 'hoher Dienstlei...",1,Ihre SoftSkill>Arbeitsweise ist proaktiv/SoftS...,"['Arbeitsweise ist proaktiv', 'hoher Dienstlei...","[Arbeitsweise ist proaktiv, hoher Dienstleistu...","[Arbeitsweise ist proaktiv, hoher Dienstleistu..."
2,1655,Voraussetzungen sind eine abgeschlossene Volk...,Voraussetzungen sind eine abgeschlossene Volk...,HMDC,"['psychische Gesundheit', 'physische Gesundhei...",1,Voraussetzungen sind eine abgeschlossene Volks...,"['psychische Gesundheit', 'physische Gesundhei...","[psychische Gesundheit, physische Gesundheit, ...","[psychische Gesundheit, physische Gesundheit, ..."
3,1252,Sie <SoftSkill_C>wertschätzen</SoftSkill_C> un...,Sie <SoftSkill>wertschätzen die Erfahrung der ...,MWA,['wertschätzen die Erfahrung der Betroffenen u...,1,Sie SoftSkill>wertschätzen die Erfahrung der B...,['wertschätzen die Erfahrung der Betroffenen u...,[wertschätzen die Erfahrung der Betroffenen un...,[wertschätzen die Erfahrung der Betroffenen un...


## GET TEST DATA 

In [3]:

import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge


# TESTING SAVED MODELS

In [4]:
import pandas as pd
import re
pd.set_option("display.max_colwidth", None)

# results_FLAN_T5_XXL = pd.read_csv("/home/user/ksharma/ks_thesis/ma-thesis/bloom_peft_lora/bloom_7bn_results_ORIG_15epochs.csv")
# results_FLAN_T5_XXL = results_FLAN_T5_XXL[results_FLAN_T5_XXL['valid'] == 1]
# results_FLAN_T5_XXL = results_FLAN_T5_XXL.drop(columns='Unnamed: 0', axis = 0)
# results_FLAN_T5_XXL = results_FLAN_T5_XXL.drop(columns='expected_skills', axis = 0)
# results_FLAN_T5_XXL = results_FLAN_T5_XXL.drop(columns='generated_skills', axis = 0)

# Function to extract skills between "<SoftSkill_C>" and "</SoftSkill_C>"
def extract_skills_c(text):
    return re.findall(r'<SoftSkill_C>(.*?)<\/SoftSkill_C', text)

# Function to extract skills between "<SoftSkill>"
def extract_skills(text):
    return re.findall(r'<SoftSkill>(.*?)<\/SoftSkill', text)

# Apply the functions to 'input', 'expected_text', and 'generated_text' columns
results_FLAN_T5_XXL['input_skills_C'] = results_FLAN_T5_XXL['input'].apply(extract_skills_c)
# df['input_skills'] = df['input'].apply(extract_skills)
results_FLAN_T5_XXL['input_skills_all'] = results_FLAN_T5_XXL['input'].str.findall(r'(?:<SoftSkill>|<SoftSkill_C>)(.*?)(?:</SoftSkill|</SoftSkill_C)')
# results_FLAN_T5_XXL['expected_skills_all'] = results_FLAN_T5_XXL['expected_text'].str.findall(r'<SoftSkill>(.*?)<\/SoftSkill')
# results_FLAN_T5_XXL['generated_skills_all'] = results_FLAN_T5_XXL['generated_text'].str.findall(r'SoftSkill>(.*?)/SoftSkill>')
results_FLAN_T5_XXL


,orig_idx,input,expected_text,Problem type,expected_skills,valid,generated_text,generated_skills,expected_skills_all,generated_skills_all,input_skills_C,input_skills_all
0,752,"Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill_C>dienstleitungsorientierten</SoftSkill_C>, <SoftSkill_C>verantwortungsbewussten</SoftSkill_C> wie <SoftSkill>teamorientierten handeln</SoftSkill>.","Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill>dienstleitungsorientierten handeln</SoftSkill>, <SoftSkill>verantwortungsbewussten handeln</SoftSkill> wie <SoftSkill>teamorientierten handeln</SoftSkill>.",MC,"['dienstleitungsorientierten handeln', 'verantwortungsbewussten handeln', 'teamorientierten handeln']",1,"Wir wenden uns an einen gelernten Uhrmacher mit einem SoftSkill>dienstleitungsorientierten handeln/SoftSkill>, SoftSkill>verantwortungsbewussten handeln/SoftSkill> wie SoftSkill>teamorientierten handeln/SoftSkill>.","['dienstleitungsorientierten handeln', 'verantwortungsbewussten handeln', 'teamorientierten handeln']","[dienstleitungsorientierten handeln, verantwortungsbewussten handeln, teamorientierten handeln]","[dienstleitungsorientierten handeln, verantwortungsbewussten handeln, teamorientierten handeln]","[dienstleitungsorientierten, verantwortungsbewussten]","[dienstleitungsorientierten, verantwortungsbewussten, teamorientierten handeln]"
1,765,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill_C>hoher Dienstleistungs-</SoftSkill_C> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill>hoher Dienstleistungsbereitschaft</SoftSkill> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,HOWA,"['Arbeitsweise ist proaktiv', 'hoher Dienstleistungsbereitschaft', 'Einsatzbereitschaft']",1,Ihre SoftSkill>Arbeitsweise ist proaktiv/SoftSkill> mit SoftSkill>hoher Dienstleistungsbereitschaft/SoftSkill> und SoftSkill>Einsatzbereitschaft/SoftSkill>.,"['Arbeitsweise ist proaktiv', 'hoher Dienstleistungsbereitschaft', 'Einsatzbereitschaft']","[Arbeitsweise ist proaktiv, hoher Dienstleistungsbereitschaft, Einsatzbereitschaft]","[Arbeitsweise ist proaktiv, hoher Dienstleistungsbereitschaft, Einsatzbereitschaft]",[hoher Dienstleistungs-],"[Arbeitsweise ist proaktiv, hoher Dienstleistungs-, Einsatzbereitschaft]"
2,1655,"Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill_C>psychische</SoftSkill_C> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill_C>Team-</SoftSkill_C> und <SoftSkill>Kommunikations-Fähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.","Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill>psychische Gesundheit</SoftSkill> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill>Teamfähigkeit</SoftSkill> und <SoftSkill>Kommunikationsfähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.",HMDC,"['psychische Gesundheit', 'physische Gesundheit', 'Kontaktfreudigkeit', 'gute Auffassungsgabe', 'Teamfähigkeit', 'Kommunikationsfähigkeit', 'praktisches Verständnis', 'manuelles Geschick']",1,"Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), SoftSkill>psychische Gesundheit/SoftSkill> und SoftSkill>physische Gesundheit/SoftSkill>, SoftSkill>Kontaktfreudigkeit/SoftSkill>, eine SoftSkill>gute Auffassungsgabe/SoftSkill>, SoftSkill>Teamfähigkeit/SoftSkill> und SoftSkill>Kommunikationsfähigkeit/SoftSkill>, SoftSkill>praktisches Verständnis/SoftSkill> und SoftSkill>manuelles Geschick/SoftSkill>.","['psychische Gesundheit', 'physische Gesundheit', 'Kontaktfreudigkeit', 'gute Auffassungsgabe', 'Teamfähigkeit', 'Kommunikationsfähigkeit', 'praktisch

In [5]:
results_FLAN_T5_XXL[results_FLAN_T5_XXL['orig_idx'] == 1910]

,orig_idx,input,expected_text,Problem type,expected_skills,valid,generated_text,generated_skills,expected_skills_all,generated_skills_all,input_skills_C,input_skills_all
29,1910,Abgeschlossene Berufslehre als Gärtner/in\nErfahrung in einer vergleichbaren Position von Vorteil\n<SoftSkill>Freude an Verkauf</SoftSkill> und <SoftSkill_C>Kundenberatung</SoftSkill_C>\n<SoftSkill_C>Engagierte</SoftSkill_C> und <SoftSkill>belastbare Persönlichkeit</SoftSkill>\n<SoftSkill>Ausgeprägte Teamfähigkeit</SoftSkill> und <SoftSkill>Kontaktfreudigkeit</SoftSkill,Abgeschlossene Berufslehre als Gärtner/in\nErfahrung in einer vergleichbaren Position von Vorteil\n<SoftSkill>Freude an Verkauf</SoftSkill> und <SoftSkill>Freude an Kundenberatung</SoftSkill>\n<SoftSkill>Engagierte Persönlichkeit</SoftSkill> und <SoftSkill>belastbare Persönlichkeit</SoftSkill>\n<SoftSkill>Ausgeprägte Teamfähigkeit</SoftSkill> und <SoftSkill>Kontaktfreudigkeit</SoftSkill>,MDC,"['Freude an Verkauf', 'Freude an Kundenberatung', 'Engagierte Persönlichkeit', 'belastbare Persönlichkeit', 'Ausgeprägte Teamfähigkeit', 'Kontaktfreudigkeit']",1,Abgeschlossene Berufslehre als Gärtner/in Erfahrung in einer vergleichbaren Position von Vorteil SoftSkill>Freude an Verkauf/SoftSkill> und SoftSkill>Freude an Kundenberatung/SoftSkill> SoftSkill>Engagierte Persönlichkeit/SoftSkill> und SoftSkill>belastbare Persönlichkeit/SoftSkill> SoftSkill>Ausgeprägte Teamfähigkeit/SoftSkill> und SoftSkill>Kontaktfreudigkeit/SoftSkill>,"['Freude an Verkauf', 'Freude an Kundenberatung', 'Engagierte Persönlichkeit', 'belastbare Persönlichkeit', 'Ausgeprägte Teamfähigkeit', 'Kontaktfreudigkeit']","[Freude an Verkauf, Freude an Kundenberatung, Engagierte Persönlichkeit, belastbare Persönlichkeit, Ausgeprägte Teamfähigkeit, Kontaktfreudigkeit]","[Freude an Verkauf, Freude an Kundenberatung, Engagierte Persönlichkeit, belastbare Persönlichkeit, Ausgeprägte Teamfähigkeit, Kontaktfreudigkeit]","[Kundenberatung, Engagierte]","[Freude an Verkauf, Kundenberatung, Engagierte, belastbare Persönlichkeit, Ausgeprägte Teamfähigkeit, Kontaktfreudigkeit]"


In [6]:
# Function to find the corresponding skills in 'expected_text' and 'generated_text'
def find_C_expected_skills(row):
    corresponding_skills = []
    for skill_c in row['input_skills_C']:
        if skill_c in row['input_skills_all']:
            idx = row['input_skills_all'].index(skill_c)
            corresponding_skills.append(row['expected_skills_all'][idx])
            # corresponding_skills.append(row['generated_skills_new'][idx])
    return corresponding_skills

# def find_C_generated_skills(row):
#     corresponding_skills = []
#     print(row['orig_idx'])
#     # if (len(row['generated_skills_all']) == len(row['input_skills_all'])):
#     for skill_c in row['input_skills_C']:
#         if skill_c in row['input_skills_all']:
#             idx = row['input_skills_all'].index(skill_c)
#             corresponding_skills.append(row['generated_skills_all'][idx])
#             # corresponding_skills.append(row['generated_skills_new'][idx])
#     return corresponding_skills
def find_C_generated_skills(row):
    corresponding_skills = []
    for skill_c, generated_skill in zip(row['input_skills_C'], row['generated_skills_all']):
        if skill_c in row['input_skills_all']:
            idx = row['input_skills_all'].index(skill_c)
            corresponding_skills.append(generated_skill)
    return corresponding_skills
# Apply the function to create 'corresponding_skills' column
results_FLAN_T5_XXL['expected_skills_C'] = results_FLAN_T5_XXL.apply(find_C_expected_skills, axis=1)
results_FLAN_T5_XXL['generated_skills_C'] = results_FLAN_T5_XXL.apply(find_C_generated_skills, axis=1)


# results_FLAN_T5_XXL[results_FLAN_T5_XXL['generated_skills_C'] == []]

In [7]:
# results_FLAN_T5_XXL['num_skills_all'] = results_FLAN_T5_XXL.apply(lambda row: len(row['input_skills_all']))
# results_FLAN_T5_XXL['num_skills_C'] = results_FLAN_T5_XXL.apply(lambda row: len(row['input_skills_C']))
results_FLAN_T5_XXL['num_skills_all'] = results_FLAN_T5_XXL.apply(lambda row: len(row['expected_skills_all']) if isinstance(row['expected_skills_all'], list) else 0, axis=1)
results_FLAN_T5_XXL['num_skills_C'] = results_FLAN_T5_XXL.apply(lambda row: len(row['expected_skills_C']) if isinstance(row['expected_skills_C'], list) else 0, axis=1)

results_FLAN_T5_XXL['num_gen_skills_all'] = results_FLAN_T5_XXL.apply(lambda row: len(row['generated_skills_all']) if isinstance(row['generated_skills_all'], list) else 0, axis=1)
results_FLAN_T5_XXL['num_gen_skills_C'] = results_FLAN_T5_XXL.apply(lambda row: len(row['generated_skills_C']) if isinstance(row['generated_skills_C'], list) else 0, axis=1)


results_FLAN_T5_XXL.head(3)

,orig_idx,input,expected_text,Problem type,expected_skills,valid,generated_text,generated_skills,expected_skills_all,generated_skills_all,input_skills_C,input_skills_all,expected_skills_C,generated_skills_C,num_skills_all,num_skills_C,num_gen_skills_all,num_gen_skills_C
0,752,"Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill_C>dienstleitungsorientierten</SoftSkill_C>, <SoftSkill_C>verantwortungsbewussten</SoftSkill_C> wie <SoftSkill>teamorientierten handeln</SoftSkill>.","Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill>dienstleitungsorientierten handeln</SoftSkill>, <SoftSkill>verantwortungsbewussten handeln</SoftSkill> wie <SoftSkill>teamorientierten handeln</SoftSkill>.",MC,"['dienstleitungsorientierten handeln', 'verantwortungsbewussten handeln', 'teamorientierten handeln']",1,"Wir wenden uns an einen gelernten Uhrmacher mit einem SoftSkill>dienstleitungsorientierten handeln/SoftSkill>, SoftSkill>verantwortungsbewussten handeln/SoftSkill> wie SoftSkill>teamorientierten handeln/SoftSkill>.","['dienstleitungsorientierten handeln', 'verantwortungsbewussten handeln', 'teamorientierten handeln']","[dienstleitungsorientierten handeln, verantwortungsbewussten handeln, teamorientierten handeln]","[dienstleitungsorientierten handeln, verantwortungsbewussten handeln, teamorientierten handeln]","[dienstleitungsorientierten, verantwortungsbewussten]","[dienstleitungsorientierten, verantwortungsbewussten, teamorientierten handeln]","[dienstleitungsorientierten handeln, verantwortungsbewussten handeln]","[dienstleitungsorientierten handeln, verantwortungsbewussten handeln]",3,2,3,2
1,765,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill_C>hoher Dienstleistungs-</SoftSkill_C> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill>hoher Dienstleistungsbereitschaft</SoftSkill> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,HOWA,"['Arbeitsweise ist proaktiv', 'hoher Dienstleistungsbereitschaft', 'Einsatzbereitschaft']",1,Ihre SoftSkill>Arbeitsweise ist proaktiv/SoftSkill> mit SoftSkill>hoher Dienstleistungsbereitschaft/SoftSkill> und SoftSkill>Einsatzbereitschaft/SoftSkill>.,"['Arbeitsweise ist proaktiv', 'hoher Dienstleistungsbereitschaft', 'Einsatzbereitschaft']","[Arbeitsweise ist proaktiv, hoher Dienstleistungsbereitschaft, Einsatzbereitschaft]","[Arbeitsweise ist proaktiv, hoher Dienstleistungsbereitschaft, Einsatzbereitschaft]",[hoher Dienstleistungs-],"[Arbeitsweise ist proaktiv, hoher Dienstleistungs-, Einsatzbereitschaft]",[hoher Dienstleistungsbereitschaft],[Arbeitsweise ist proaktiv],3,1,3,1
2,1655,"Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill_C>psychische</SoftSkill_C> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill_C>Team-</SoftSkill_C> und <SoftSkill>Kommunikations-Fähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.","Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill>psychische Gesundheit</SoftSkill> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill>Teamfähigkeit</SoftSkill> und <SoftSkill>Kommunikationsfähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.",HMDC,"['psychische Gesundheit', 'physische Gesundheit', 'Kontaktfreudigkeit', 'gute Auffassungsgabe', 'Teamfähigkeit', 'Kommunikationsfähigkeit', 'praktisches Verständnis', 'manuelles Geschick']",1,"Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), SoftSkill>psychische Gesundheit/SoftSkill> und SoftSkill>physische Gesundheit/SoftSkill>, SoftSkill>Kontaktfreudigkeit/SoftSkill>, eine SoftSkill>gute Auffassungsgabe/SoftSkill>, So

In [153]:
# results_FLAN_T5_XXL.loc[results_FLAN_T5_XXL['num_skills_C'] == 0, 'input'] = results_FLAN_T5_XXL.loc[results_FLAN_T5_XXL['num_skills_C'] == 0, 'input'].apply(lambda x: x + '>')

In [8]:
results_FLAN_T5_XXL[results_FLAN_T5_XXL['num_skills_C']==0]

,orig_idx,input,expected_text,Problem type,expected_skills,valid,generated_text,generated_skills,expected_skills_all,generated_skills_all,input_skills_C,input_skills_all,expected_skills_C,generated_skills_C,num_skills_all,num_skills_C,num_gen_skills_all,num_gen_skills_C


In [9]:

from collections import Counter

# Create a new column 'skills_match_score'
results_FLAN_T5_XXL['percent_skills_match_all'] = 0.0
results_FLAN_T5_XXL['percent_skills_match_C'] = 0.0

results_FLAN_T5_XXL['num_skills_match_all'] = 0.0
results_FLAN_T5_XXL['num_skills_match_C'] = 0.0

# Iterate over the rows of the DataFrame
for index, row in results_FLAN_T5_XXL.iterrows():
    expected_skills_all = set(row['expected_skills_all'])
    generated_skills_all = set(row['generated_skills_all'])
    
    expected_skills_C = set(row['expected_skills_C'])
    generated_skills_C = set(row['generated_skills_C'])

    count_gen_C = Counter(generated_skills_C)
    count_exp_C = Counter(expected_skills_C)

    # Find the common elements between list1 and list2
    common_elements_C = set(expected_skills_C).intersection(set(generated_skills_C))

    # Find the number of common elements, considering duplicates
    num_common_elements_C = sum(min(count_gen_C[element], count_exp_C[element]) for element in common_elements_C)

    count_gen_all = Counter(generated_skills_all)
    count_exp_all = Counter(expected_skills_all)

    # Find the common elements between list1 and list2
    common_elements_all = set(expected_skills_all).intersection(set(generated_skills_all))

    # Find the number of common elements, considering duplicates
    num_common_elements_all = sum(min(count_gen_all[element], count_exp_all[element]) for element in common_elements_all)



    # Calculate the percentage of matching skills
    if len(expected_skills_all) > 0:
        match_score_all = len(expected_skills_all.intersection(generated_skills_all)) / len(expected_skills_all) * 100
    else:
        match_score_all = 0.0
    # Calculate the percentage of matching skills
    if len(expected_skills_C) > 0:
        match_score_C = len(expected_skills_C.intersection(generated_skills_C)) / len(expected_skills_C) * 100
    else:
        match_score_C = 0.0
    
    # Assign the match score to the 'skills_match_score' column
    results_FLAN_T5_XXL.at[index, 'percent_skills_match_all'] = match_score_all
    results_FLAN_T5_XXL.at[index, 'percent_skills_match_C'] = match_score_C
    results_FLAN_T5_XXL.at[index, 'num_skills_match_all'] = num_common_elements_all
    results_FLAN_T5_XXL.at[index, 'num_skills_match_C'] = num_common_elements_C

In [10]:
results_FLAN_T5_XXL[results_FLAN_T5_XXL['percent_skills_match_all'] == 100]['Problem type'].value_counts()

Problem type
OWA     49
MWA     42
MC      28
HOWA    14
MDC      7
HMDC     5
NONE     3
HMC      1
Name: count, dtype: int64

In [11]:
results_FLAN_T5_XXL[results_FLAN_T5_XXL['percent_skills_match_all'] == 100]['Problem type'].value_counts()

Problem type
OWA     49
MWA     42
MC      28
HOWA    14
MDC      7
HMDC     5
NONE     3
HMC      1
Name: count, dtype: int64

In [12]:
import Levenshtein
import re 
def remove_sp_char(sent):
    modified_text = re.sub(r'(<)?(?:\/?(?:SoftSkill(?:_C)?))?(>)', '', sent)
    modified_text = re.sub(r'\n', ' ', modified_text)
    modified_text = re.sub(r'\r', ' ', modified_text)
    return modified_text

results_FLAN_T5_XXL['cleaned_generated_text'] = results_FLAN_T5_XXL['generated_text'].apply(remove_sp_char)
results_FLAN_T5_XXL['cleaned_expected_text'] = results_FLAN_T5_XXL['expected_text'].apply(remove_sp_char)
results_FLAN_T5_XXL['sentence_levenshtein'] = results_FLAN_T5_XXL.apply(lambda row: Levenshtein.distance(row['cleaned_generated_text'], row['cleaned_expected_text']), axis=1)
# results_FLAN_T5_XXL['skills_levenshtein'] = results_FLAN_T5_XXL.apply(lambda row: Levenshtein.distance(str(row['generated_skills']), str(row['expected_skills'])), axis=1)

In [13]:
results_FLAN_T5_XXL[results_FLAN_T5_XXL['orig_idx']==29]

,orig_idx,input,expected_text,Problem type,expected_skills,valid,generated_text,generated_skills,expected_skills_all,generated_skills_all,...,num_skills_C,num_gen_skills_all,num_gen_skills_C,percent_skills_match_all,percent_skills_match_C,num_skills_match_all,num_skills_match_C,cleaned_generated_text,cleaned_expected_text,sentence_levenshtein
106,29,<SoftSkill_C>Selbständige</SoftSkill_C> und <SoftSkill>zuverlässige Arbeitsweise</SoftSkill,<SoftSkill>Selbständige Arbeitsweise</SoftSkill> und <SoftSkill>zuverlässige Arbeitsweise</SoftSkill>,OWA,"['Selbständige Arbeitsweise', 'zuverlässige Arbeitsweise']",1,SoftSkill>Selbständige Arbeitsweise/SoftSkill> und SoftSkill>zuverlässige Arbeitsweise/SoftSkill>,"['Selbständige Arbeitsweise', 'zuverlässige Arbeitsweise']","[Selbständige Arbeitsweise, zuverlässige Arbeitsweise]","[Selbständige Arbeitsweise, zuverlässige Arbeitsweise]",...,1,2,1,100.0,100.0,2.0,1.0,Selbständige Arbeitsweise und zuverlässige Arbeitsweise,Selbständige Arbeitsweise und zuverlässige Arbeitsweise,0


In [14]:
from rouge import Rouge
# Initialize the ROUGE scorer

rouge = Rouge()
rouge_score = rouge.get_scores("['wertschätzend', 'respektvoll miteinader kommunizieren', 'konstruktiv im Interesse']", 
                                "['wertschätzend', 'respektvoll miteinader zu kommunizieren', 'konstruktiv im Interesse']")[0]
rouge_score



{'rouge-1': {'r': 0.875, 'p': 1.0, 'f': 0.9333333283555556},
 'rouge-2': {'r': 0.7142857142857143,
  'p': 0.8333333333333334,
  'f': 0.7692307642603551},
 'rouge-l': {'r': 0.875, 'p': 1.0, 'f': 0.9333333283555556}}

In [15]:
# Define functions to calculate Rouge and BLEU scores
def calculate_rouge_scores(expected_list, generated_list):
    rouge = Rouge()
    rouge_scores = []
    min_length = min(len(expected_list), len(generated_list))
    for i in range(min_length):
        expected = expected_list[i]
        generated = generated_list[i]
        scores = rouge.get_scores(expected, generated)
        rouge_scores.append(scores)
    return rouge_scores

def calculate_bleu_score(expected_list, generated_list):
    smoothing = SmoothingFunction().method0
    bleu_scores = []
    for expected, generated in zip(expected_list, generated_list):
        if len(expected_list) == len(generated_list):
            bleu_score = sentence_bleu([expected], generated, smoothing_function=smoothing)
            bleu_scores.append(bleu_score)
    return bleu_scores

def calculate_levenshtein_distance(expected_list, generated_list):
    levenshtein_distances = []
    for expected, generated in zip(expected_list, generated_list):
        if len(expected_list) == len(generated_list):
            distance = Levenshtein.distance(expected, generated)
            levenshtein_distances.append(distance)
    return levenshtein_distances

# Calculate the Rouge and BLEU scores for each row and store them in new columns
results_FLAN_T5_XXL['rouge_scores_skill_C'] = results_FLAN_T5_XXL.apply(lambda row: calculate_rouge_scores(row['expected_skills_C'], row['generated_skills_C']), axis=1)
results_FLAN_T5_XXL['bleu_scores_skill_C'] = results_FLAN_T5_XXL.apply(lambda row: calculate_bleu_score(row['expected_skills_C'], row['generated_skills_C']), axis=1)

# Calculate the average scores for each pair individually
results_FLAN_T5_XXL['rouge1_avg_skill_C'] = results_FLAN_T5_XXL['rouge_scores_skill_C'].apply(lambda x: sum(score[0]['rouge-1']['f'] for score in x) / len(x) if len(x) > 0 else-1)
results_FLAN_T5_XXL['rouge2_avg_skill_C'] = results_FLAN_T5_XXL['rouge_scores_skill_C'].apply(lambda x: sum(score[0]['rouge-2']['f'] for score in x) / len(x) if len(x) > 0 else -1)
results_FLAN_T5_XXL['rougeL_avg_skill_C'] = results_FLAN_T5_XXL['rouge_scores_skill_C'].apply(lambda x: sum(score[0]['rouge-l']['f'] for score in x) / len(x) if len(x) > 0 else -1)
results_FLAN_T5_XXL['bleu_avg_skill_C'] = results_FLAN_T5_XXL['bleu_scores_skill_C'].apply(lambda x: sum(x) / len(x) if len(x) > 0 else -1)

# Calculate the Levenshtein distances for each pair individually and store the sum in a new column
results_FLAN_T5_XXL['levenshtein_skill_C'] = results_FLAN_T5_XXL.apply(lambda row: sum(calculate_levenshtein_distance(row['expected_skills_C'], row['generated_skills_C'])), axis=1)


# Calculate the Rouge and BLEU scores for each row and store them in new columns
results_FLAN_T5_XXL['rouge_scores_skill_all'] = results_FLAN_T5_XXL.apply(lambda row: calculate_rouge_scores(row['expected_skills_all'], row['generated_skills_all']), axis=1)
results_FLAN_T5_XXL['bleu_scores_skill_all'] = results_FLAN_T5_XXL.apply(lambda row: calculate_bleu_score(row['expected_skills_all'], row['generated_skills_all']), axis=1)

# Calculate the average scores for each pair individually
results_FLAN_T5_XXL['rouge1_avg_skill_all'] = results_FLAN_T5_XXL['rouge_scores_skill_all'].apply(lambda x: sum(score[0]['rouge-1']['f'] for score in x) / len(x) if len(x) > 0 else -1)
results_FLAN_T5_XXL['rouge2_avg_skill_all'] = results_FLAN_T5_XXL['rouge_scores_skill_all'].apply(lambda x: sum(score[0]['rouge-2']['f'] for score in x) / len(x) if len(x) > 0 else -1)
results_FLAN_T5_XXL['rougeL_avg_skill_all'] = results_FLAN_T5_XXL['rouge_scores_skill_all'].apply(lambda x: sum(score[0]['rouge-l']['f'] for score in x) / len(x) if len(x) > 0 else -1)
results_FLAN_T5_XXL['bleu_avg_skill_all'] = results_FLAN_T5_XXL['bleu_scores_skill_all'].apply(lambda x: sum(x) / len(x) if len(x) > 0 else -1)

# Calculate the Levenshtein distances for each pair individually and store the sum in a new column
results_FLAN_T5_XXL['levenshtein_skill_all'] = results_FLAN_T5_XXL.apply(lambda row: sum(calculate_levenshtein_distance(row['expected_skills_all'], row['generated_skills_all'])), axis=1)

results_FLAN_T5_XXL[results_FLAN_T5_XXL['orig_idx']==694]



/home/user/ksharma/miniconda3/envs/ks_tars_thesis/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/user/ksharma/miniconda3/envs/ks_tars_thesis/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/user/ksharma/miniconda3/envs/ks_tars_thesis/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram o

,orig_idx,input,expected_text,Problem type,expected_skills,valid,generated_text,generated_skills,expected_skills_all,generated_skills_all,...,rougeL_avg_skill_C,bleu_avg_skill_C,levenshtein_skill_C,rouge_scores_skill_all,bleu_scores_skill_all,rouge1_avg_skill_all,rouge2_avg_skill_all,rougeL_avg_skill_all,bleu_avg_skill_all,levenshtein_skill_all
5,694,"Mit Ihrer <SoftSkill_C>offenen</SoftSkill_C> und <SoftSkill>positiven Art</SoftSkill> bedienen Sie unsere Gäste freundlich, <SoftSkill_C>schnell</SoftSkill_C> und nach STARBUCKS® Standards.","Mit Ihrer <SoftSkill>offenen Art</SoftSkill> und <SoftSkill>positiven Art</SoftSkill> bedienen Sie unsere Gäste freundlich, <SoftSkill>bedienen Sie unsere Gäste schnell</SoftSkill> und nach STARBUCKS® Standards.",MDC,"['offenen Art', 'positiven Art', 'bedienen Sie unsere Gäste schnell']",1,"Mit Ihrer SoftSkill>offenen Art/SoftSkill> und SoftSkill>positiven Art/SoftSkill> bedienen Sie unsere Gäste freundlich, SoftSkill>schnell/SoftSkill> und nach STARBUCKS® Standards.","['offenen Art', 'positiven Art', 'schnell']","[offenen Art, positiven Art, bedienen Sie unsere Gäste schnell]","[offenen Art, positiven Art, schnell]",...,0.5,0.5,28,"[[{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}], [{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}], [{'rouge-1': {'r': 1.0, 'p': 0.2, 'f': 0.33333333055555564}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 1.0, 'p': 0.2, 'f': 0.33333333055555564}}]]","[1.0, 1.0, 0.0243728440732796]",0.777778,0.666667,0.777778,0.674791,26


In [16]:
results_FLAN_T5_XXL.columns

Index(['orig_idx', 'input', 'expected_text', 'Problem type', 'expected_skills',
       'valid', 'generated_text', 'generated_skills', 'expected_skills_all',
       'generated_skills_all', 'input_skills_C', 'input_skills_all',
       'expected_skills_C', 'generated_skills_C', 'num_skills_all',
       'num_skills_C', 'num_gen_skills_all', 'num_gen_skills_C',
       'percent_skills_match_all', 'percent_skills_match_C',
       'num_skills_match_all', 'num_skills_match_C', 'cleaned_generated_text',
       'cleaned_expected_text', 'sentence_levenshtein', 'rouge_scores_skill_C',
       'bleu_scores_skill_C', 'rouge1_avg_skill_C', 'rouge2_avg_skill_C',
       'rougeL_avg_skill_C', 'bleu_avg_skill_C', 'levenshtein_skill_C',
       'rouge_scores_skill_all', 'bleu_scores_skill_all',
       'rouge1_avg_skill_all', 'rouge2_avg_skill_all', 'rougeL_avg_skill_all',
       'bleu_avg_skill_all', 'levenshtein_skill_all'],
      dtype='object')

In [164]:
# results_FLAN_T5_XXL.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/bloom-7b1-lora-ORIG-stopseq20230717114608.csv")


In [17]:
results_FLAN_T5_XXL['Problem type'].value_counts()

Problem type
OWA     63
MWA     53
MC      34
HOWA    19
MDC     11
HMDC    10
NONE     3
HMC      1
Name: count, dtype: int64

In [166]:
# results_FLAN_T5_XXL[results_FLAN_T5_XXL['skills_match_score'] == 100]['Problem type'].value_counts()

In [168]:
# import re 
# def remove_sp_char(sent):
#     modified_text = re.sub(r'(<)?(?:\/?(?:SoftSkill(?:_C)?))?(>)', '', sent)
#     modified_text = re.sub(r'\n', ' ', modified_text)
#     return modified_text

# results_FLAN_T5_XXL['cleaned_generated_text'] = results_FLAN_T5_XXL['generated_text'].apply(remove_sp_char)

In [169]:
# results_FLAN_T5_XXL['input_C'] = results_FLAN_T5_XXL.apply(lambda row: Levenshtein.distance(row['cleaned_generated_text'], row['cleaned_expected_text']), axis=1)

In [18]:
pd.set_option('display.max_columns', None)  # Display all columns
# results_FLAN_T5_XXL[results_FLAN_T5_XXL['num_skills_C'] != results_FLAN_T5_XXL['num_gen_skills_C']]
results_FLAN_T5_XXL[results_FLAN_T5_XXL['num_skills_all'] != results_FLAN_T5_XXL['num_gen_skills_all']]
results_FLAN_T5_XXL['mismatched_ALL'] = results_FLAN_T5_XXL['num_skills_all'] != results_FLAN_T5_XXL['num_gen_skills_all']
results_FLAN_T5_XXL[results_FLAN_T5_XXL['num_skills_all'] != results_FLAN_T5_XXL['num_gen_skills_all']].head(3)

,orig_idx,input,expected_text,Problem type,expected_skills,valid,generated_text,generated_skills,expected_skills_all,generated_skills_all,input_skills_C,input_skills_all,expected_skills_C,generated_skills_C,num_skills_all,num_skills_C,num_gen_skills_all,num_gen_skills_C,percent_skills_match_all,percent_skills_match_C,num_skills_match_all,num_skills_match_C,cleaned_generated_text,cleaned_expected_text,sentence_levenshtein,rouge_scores_skill_C,bleu_scores_skill_C,rouge1_avg_skill_C,rouge2_avg_skill_C,rougeL_avg_skill_C,bleu_avg_skill_C,levenshtein_skill_C,rouge_scores_skill_all,bleu_scores_skill_all,rouge1_avg_skill_all,rouge2_avg_skill_all,rougeL_avg_skill_all,bleu_avg_skill_all,levenshtein_skill_all,mismatched_ALL
52,990,"• <SoftSkill>Interesse am Detailhandel</SoftSkill> und der <SoftSkill_C>Mode</SoftSkill_C>\nWir wenden uns an junge, <SoftSkill>motivierte Persönlichkeiten</SoftSkill>, die <SoftSkill>gerne im Team arbeiten</SoftSkill> und <SoftSkill>verantwortungsbewusst\nsind</SoftSkill>.","• <SoftSkill>Interesse am Detailhandel</SoftSkill> und <SoftSkill>Interesse an der Mode</SoftSkill>\nWir wenden uns an junge, <SoftSkill>motivierte Persönlichkeiten</SoftSkill>, die <SoftSkill>gerne im Team arbeiten</SoftSkill> und <SoftSkill>verantwortungsbewusst\nsind</SoftSkill>.",MWA,"['Interesse am Detailhandel', 'Interesse an der Mode', 'motivierte Persönlichkeiten', 'gerne im Team arbeiten']",1,"• SoftSkill>Interesse am Detailhandel/SoftSkill> und SoftSkill>Interesse am Modehandel/SoftSkill> Wir wenden uns an junge, SoftSkill>motivierte Persönlichkeiten/SoftSkill>, die SoftSkill>gerne im Team arbeiten/SoftSkill> und SoftSkill>verantwortungsbewusst sind/SoftSkill>.","['Interesse am Detailhandel', 'Interesse am Modehandel', 'motivierte Persönlichkeiten', 'gerne im Team arbeiten', 'verantwortungsbewusst sind']","[Interesse am Detailhandel, Interesse an der Mode, motivierte Persönlichkeiten, gerne im Team arbeiten]","[Interesse am Detailhandel, Interesse am Modehandel, motivierte Persönlichkeiten, gerne im Team arbeiten, verantwortungsbewusst sind]",[Mode],"[Interesse am Detailhandel, Mode, motivierte Persönlichkeiten, gerne im Team arbeiten]",[Interesse an der Mode],[Interesse am Detailhandel],4,1,5,1,75.0,0.000000,3.0,0.0,"• Interesse am Detailhandel und Interesse am Modehandel Wir wenden uns an junge, motivierte Persönlichkeiten, die gerne im Team arbeiten und verantwortungsbewusst sind.","• Interesse am Detailhandel und Interesse an der Mode Wir wenden uns an junge, motivierte Persönlichkeiten, die gerne im Team arbeiten und verantwortungsbewusst sind.",9,"[[{'rouge-1': {'r': 0.3333333333333333, 'p': 0.25, 'f': 0.2857142808163266}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.3333333333333333, 'p': 0.25, 'f': 0.2857142808163266}}]]",[0.4619372552064899],0.285714,0.0,0.285714,0.461937,10,"[[{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}], [{'rouge-1': {'r': 0.3333333333333333, 'p': 0.25, 'f': 0.2857142808163266}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.3333333333333333, 'p': 0.25, 'f': 0.2857142808163266}}], [{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}], [{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}]]",[],0.821429,0.750000,0.821429,-1.0,0,True
60,507,Sie <SoftSkill_C>denken</SoftSkill_C> und <SoftSkill>handeln kunden-</SoftSkill> und <SoftSkill_C>qualitätsorientiert</SoftSkill_C>.,Sie <SoftSkill>denken kundenorientiert</SoftSkill> und <SoftSkill>handeln kundenorientiert</SoftSkill> und <SoftSkill>denken qualitätsorientiert</SoftSkill> und <SoftSkill>handeln qualitätsorientiert</SoftSkill>.,HMDC,"['denken kundenorientiert', 'handeln kundenorientiert', 'denken qualitätsorient

In [20]:
results_C = results_FLAN_T5_XXL[['orig_idx', 'input', 'expected_text', 'generated_text', 'input_skills_C', 'expected_skills_C', 'generated_skills_C',
                'num_skills_C', 'num_gen_skills_C', 'percent_skills_match_C', 'num_skills_match_C',
                'rouge_scores_skill_C', 'bleu_scores_skill_C', 'rouge1_avg_skill_C',
                'rouge2_avg_skill_C', 'rougeL_avg_skill_C', 'bleu_avg_skill_C', 'levenshtein_skill_C', 'mismatched_ALL','Problem type',
                'valid']]

# Create "results_ALL" DataFrame
results_ALL = results_FLAN_T5_XXL[['orig_idx', 'input', 'expected_text', 'generated_text', 'input_skills_all', 'expected_skills_all', 'generated_skills_all',
                  'num_skills_all', 'num_gen_skills_all', 'percent_skills_match_all', 'num_skills_match_all',
                  'rouge_scores_skill_all', 'bleu_scores_skill_all', 'rouge1_avg_skill_all',
                  'rouge2_avg_skill_all', 'rougeL_avg_skill_all', 'bleu_avg_skill_all', 'levenshtein_skill_all', 'mismatched_ALL','Problem type',
                'valid']]

# Create "results_SENTENCE" DataFrame
results_SENTENCE = results_FLAN_T5_XXL[['orig_idx', 'input', 'expected_text', 'generated_text', 'cleaned_generated_text', 'cleaned_expected_text', 'sentence_levenshtein','Problem type',
                'valid']]

# # Export the three DataFrames to separate CSV files (optional)
# results_C.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/BLOOM_results_C.csv", index=False)
# results_ALL.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/BLOOM_results_ALL.csv", index=False)
# results_SENTENCE.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/BLOOM_results_SENTENCE.csv", index=False)

In [21]:
results_ALL[results_ALL['mismatched_ALL']==True].head(5)

,orig_idx,input,expected_text,generated_text,input_skills_all,expected_skills_all,generated_skills_all,num_skills_all,num_gen_skills_all,percent_skills_match_all,num_skills_match_all,rouge_scores_skill_all,bleu_scores_skill_all,rouge1_avg_skill_all,rouge2_avg_skill_all,rougeL_avg_skill_all,bleu_avg_skill_all,levenshtein_skill_all,mismatched_ALL,Problem type,valid
52,990,"• <SoftSkill>Interesse am Detailhandel</SoftSkill> und der <SoftSkill_C>Mode</SoftSkill_C>\nWir wenden uns an junge, <SoftSkill>motivierte Persönlichkeiten</SoftSkill>, die <SoftSkill>gerne im Team arbeiten</SoftSkill> und <SoftSkill>verantwortungsbewusst\nsind</SoftSkill>.","• <SoftSkill>Interesse am Detailhandel</SoftSkill> und <SoftSkill>Interesse an der Mode</SoftSkill>\nWir wenden uns an junge, <SoftSkill>motivierte Persönlichkeiten</SoftSkill>, die <SoftSkill>gerne im Team arbeiten</SoftSkill> und <SoftSkill>verantwortungsbewusst\nsind</SoftSkill>.","• SoftSkill>Interesse am Detailhandel/SoftSkill> und SoftSkill>Interesse am Modehandel/SoftSkill> Wir wenden uns an junge, SoftSkill>motivierte Persönlichkeiten/SoftSkill>, die SoftSkill>gerne im Team arbeiten/SoftSkill> und SoftSkill>verantwortungsbewusst sind/SoftSkill>.","[Interesse am Detailhandel, Mode, motivierte Persönlichkeiten, gerne im Team arbeiten]","[Interesse am Detailhandel, Interesse an der Mode, motivierte Persönlichkeiten, gerne im Team arbeiten]","[Interesse am Detailhandel, Interesse am Modehandel, motivierte Persönlichkeiten, gerne im Team arbeiten, verantwortungsbewusst sind]",4,5,75.0,3.0,"[[{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}], [{'rouge-1': {'r': 0.3333333333333333, 'p': 0.25, 'f': 0.2857142808163266}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.3333333333333333, 'p': 0.25, 'f': 0.2857142808163266}}], [{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}], [{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}]]",[],0.821429,0.750000,0.821429,-1.0,0,True,MWA,1
60,507,Sie <SoftSkill_C>denken</SoftSkill_C> und <SoftSkill>handeln kunden-</SoftSkill> und <SoftSkill_C>qualitätsorientiert</SoftSkill_C>.,Sie <SoftSkill>denken kundenorientiert</SoftSkill> und <SoftSkill>handeln kundenorientiert</SoftSkill> und <SoftSkill>denken qualitätsorientiert</SoftSkill> und <SoftSkill>handeln qualitätsorientiert</SoftSkill>.,Sie SoftSkill>denken kundenorientiert/SoftSkill> und SoftSkill>handeln kundenorientiert/SoftSkill> und SoftSkill>handeln qualitätsorientiert/SoftSkill>.,"[denken, handeln kunden-, qualitätsorientiert]","[denken kundenorientiert, handeln kundenorientiert, denken qualitätsorientiert, handeln qualitätsorientiert]","[denken kundenorientiert, handeln kundenorientiert, handeln qualitätsorientiert]",4,3,75.0,3.0,"[[{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}], [{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}], [{'rouge-1': {'r': 0.5, 'p': 0.5, 'f': 0.4999999950000001}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.5, 'p': 0.5, 'f': 0.4999999950000001}}]]",[],0.833333,0.666667,0.833333,-1.0,0,True,HMDC,1
145,483,"Naturwissenschaftliche Ausbildung (Master Degree)\nProfunder Leistungsausweis im internationalen Produktmanagement\nPraktische Marketingerfahrung von Vorteil\n<SoftSkill_C>Kommunikations-</SoftSkill_C> und <SoftSkill>Präsentationsflair</SoftSkill> (e/d) sowie Projektmanagementerfahrung\n<SoftSkill_C>Zielfokussierte</SoftSkill_C>, <SoftSkill_C>analytische Denk-</SoftSkill_C> und <SoftSkill>Handlungsweise</SoftSkill","Naturwi

In [22]:
pd.set_option('display.max_row', None)
results_C['levenshtein_skill_C'].mean()

15.435897435897436

## Sentence ROUGE

In [23]:
from rouge import Rouge

# Function to calculate Rouge scores
def calculate_rouge_scores(reference, candidate):
    rouge = Rouge()
    scores = rouge.get_scores(reference, candidate)
    rouge_1_f = scores[0]['rouge-1']['f'] if 'rouge-1' in scores[0] else None
    rouge_2_f = scores[0]['rouge-2']['f'] if 'rouge-2' in scores[0] else None
    rouge_l_f = scores[0]['rouge-l']['f'] if 'rouge-l' in scores[0] else None
    return rouge_1_f, rouge_2_f, rouge_l_f

# Apply the functions to calculate the scores and add them as new columns
results_SENTENCE['rouge1_score'], results_SENTENCE['rouge2_score'], results_SENTENCE['rougeL_score'] = zip(*results_SENTENCE.apply(
    lambda row: calculate_rouge_scores(row['cleaned_expected_text'], row['cleaned_generated_text']),
    axis=1
))

/tmp/ipykernel_282792/809110196.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_SENTENCE['rouge1_score'], results_SENTENCE['rouge2_score'], results_SENTENCE['rougeL_score'] = zip(*results_SENTENCE.apply(
/tmp/ipykernel_282792/809110196.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_SENTENCE['rouge1_score'], results_SENTENCE['rouge2_score'], results_SENTENCE['rougeL_score'] = zip(*results_SENTENCE.apply(
/tmp/ipykernel_282792/809110196.py:13: SettingWithCopyWarning: 
A value is try

## Sentence BERT Cosine Similarity

In [24]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch

# Assuming you have already read the DataFrame from the CSV file
# results_SENTENCE = pd.read_csv("/path/to/results_SENTENCE.csv")

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to calculate cosine similarity
def calculate_cosine_similarity(sentence1, sentence2):
    # Tokenize and convert sentences to tensors
    inputs = tokenizer([sentence1, sentence2], return_tensors="pt", padding=True, truncation=True)

    # Get BERT embeddings
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :].numpy()

    # Normalize embeddings
    normalized_embeddings = embeddings / (embeddings ** 2).sum(axis=-1, keepdims=True) ** 0.5

    # Calculate cosine similarity
    similarity = (normalized_embeddings[0] @ normalized_embeddings[1].T).item()
    return similarity

# Apply the function to calculate cosine similarity and add it as a new column
results_SENTENCE['cosine_similarity'] = results_SENTENCE.apply(lambda row: calculate_cosine_similarity(row['cleaned_generated_text'], row['cleaned_expected_text']), axis=1)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/tmp/ipykernel_282792/2919956664.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



## FUnction for cosine similarity for _C and ALL

In [25]:

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to get BERT embeddings for a list of sentences
def get_bert_embeddings(sentences):
    inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :].numpy()
    return embeddings

# Function to calculate cosine similarity between two lists of sentences
def calculate_cosine_similarity(list1, list2):
    if not list1 or not list2:
        return np.nan

    # Get BERT embeddings for both lists of sentences
    embeddings1 = get_bert_embeddings(list1)
    embeddings2 = get_bert_embeddings(list2)

    # Normalize embeddings
    normalized_embeddings1 = embeddings1 / (embeddings1 ** 2).sum(axis=-1, keepdims=True) ** 0.5
    normalized_embeddings2 = embeddings2 / (embeddings2 ** 2).sum(axis=-1, keepdims=True) ** 0.5

    # Calculate cosine similarity for each sentence pair
    similarity_values = np.dot(normalized_embeddings1, normalized_embeddings2.T)
    return similarity_values.diagonal().tolist()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [26]:

# Apply the function to calculate pairwise cosine similarity and store the distances in a list
cosine_similarity_list = []
for index, row in results_C.iterrows():
    list1 = row['expected_skills_C']
    list2 = row['generated_skills_C']

    # Calculate cosine similarity for each sentence pair in both lists
    pairwise_similarities = calculate_cosine_similarity(list1, list2)

    # Append the list of pairwise similarities for the row
    cosine_similarity_list.append(pairwise_similarities)

# Add the list of pairwise cosine similarity distances as a new column in the DataFrame
results_C['pairwise_cosine_similarity'] = cosine_similarity_list



# Apply the function to calculate pairwise cosine similarity and store the distances in a list
cosine_similarity_list = []
for index, row in results_ALL.iterrows():
    list1 = row['expected_skills_all']
    list2 = row['generated_skills_all']

    # Calculate cosine similarity for each sentence pair in both lists
    pairwise_similarities = calculate_cosine_similarity(list1, list2)

    # Append the list of pairwise similarities for the row
    cosine_similarity_list.append(pairwise_similarities)

# Add the list of pairwise cosine similarity distances as a new column in the DataFrame
results_ALL['pairwise_cosine_similarity'] = cosine_similarity_list

results_C['average_cosine_similarity'] = results_C['pairwise_cosine_similarity'].apply(lambda x: np.mean(x) if x else np.nan)
results_ALL['average_cosine_similarity'] = results_ALL['pairwise_cosine_similarity'].apply(lambda x: np.mean(x) if x else np.nan)


/tmp/ipykernel_282792/2476177218.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_C['pairwise_cosine_similarity'] = cosine_similarity_list
/tmp/ipykernel_282792/2476177218.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_ALL['pairwise_cosine_similarity'] = cosine_similarity_list
/tmp/ipykernel_282792/2476177218.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [157]:
# results_ALL[results_ALL['mismatched_ALL']==True].head(5)

In [27]:
results_SENTENCE.columns

Index(['orig_idx', 'input', 'expected_text', 'generated_text',
       'cleaned_generated_text', 'cleaned_expected_text',
       'sentence_levenshtein', 'Problem type', 'valid', 'rouge1_score',
       'rouge2_score', 'rougeL_score', 'cosine_similarity'],
      dtype='object')

## Prepare Metrics and Tables 

In [28]:
# Create a DataFrame for _C metrics
summary_C_df = pd.DataFrame({
    'Metric': ['Average Rouge-1 Score (C)', 'Average Rouge-2 Score (C)',
               'Average Rouge-L Score (C)',  'Average Levenshtein (C)',
               'Weighted Avg. Percentage Skills (C)', 'Cosine Similarity(C)'],
    'Value': [results_C['rouge1_avg_skill_C'].mean(), results_C['rouge2_avg_skill_C'].mean(),
              results_C['rougeL_avg_skill_C'].mean(),results_C['levenshtein_skill_C'].mean(),
              results_C['num_skills_match_C'].sum() / results_C['num_skills_C'].sum(),results_C['average_cosine_similarity'].mean()]
})

# Create a DataFrame for "ALL" metrics
summary_ALL_df = pd.DataFrame({
    'Metric': ['Average Rouge-1 Score (ALL)', 'Average Rouge-2 Score (ALL)',
               'Average Rouge-L Score (ALL)', 'Average Levenshtein (ALL)',
               'Weighted Avg. Percentage Skills (ALL)', 'Cosine Similarity(ALL)'],
    'Value': [results_ALL['rouge1_avg_skill_all'].mean(), results_ALL['rouge2_avg_skill_all'].mean(),
              results_ALL['rougeL_avg_skill_all'].mean(), results_ALL['levenshtein_skill_all'].mean(),
              results_ALL['num_skills_match_all'].sum() / results_ALL['num_skills_all'].sum(),results_ALL['average_cosine_similarity'].mean()]
})

# Create a DataFrame for "ALL" metrics
summary_SENTENCE_df = pd.DataFrame({
    'Metric': ['Average Rouge-1 Score (SENTENCE)', 'Average Rouge-2 Score (SENTENCE)',
               'Average Rouge-L Score (SENTENCE)', 'Average Levenshtein (SENTENCE)',
               'Cosine Similarity(SENTENCE)'],
    'Value': [results_SENTENCE['rouge1_score'].mean(), results_SENTENCE['rouge2_score'].mean(),
              results_SENTENCE['rougeL_score'].mean(), results_SENTENCE['sentence_levenshtein'].mean(),
              results_SENTENCE['cosine_similarity'].mean()]
})

# Export the summary DataFrames to CSV
# summary_C_df.to_csv("summary_metrics_C.csv", index=False)
# summary_ALL_df.to_csv("summary_metrics_ALL.csv", index=False)

In [29]:
summary_C_df

,Metric,Value
0,Average Rouge-1 Score (C),0.608155
1,Average Rouge-2 Score (C),0.513700
2,Average Rouge-L Score (C),0.608155
3,Average Levenshtein (C),15.435897
4,Weighted Avg. Percentage Skills (C),0.516854
5,Cosine Similarity(C),0.959026


In [30]:
summary_ALL_df

,Metric,Value
0,Average Rouge-1 Score (ALL),0.963547
1,Average Rouge-2 Score (ALL),0.838946
2,Average Rouge-L Score (ALL),0.963547
3,Average Levenshtein (ALL),3.271795
4,Weighted Avg. Percentage Skills (ALL),0.910355
5,Cosine Similarity(ALL),0.995852


In [31]:
summary_SENTENCE_df

,Metric,Value
0,Average Rouge-1 Score (SENTENCE),0.979844
1,Average Rouge-2 Score (SENTENCE),0.952241
2,Average Rouge-L Score (SENTENCE),0.979844
3,Average Levenshtein (SENTENCE),4.415385
4,Cosine Similarity(SENTENCE),0.989997


## Problem Types DATA RESULTS _C SKILLS

In [32]:
summary_C_df_list = []

# Group the DataFrame by "Problem Type"
grouped_results_C = results_C.groupby('Problem type')

# Iterate over each group and calculate the summary DataFrame
for problem_type, group in grouped_results_C:
    num_samples = len(group)  # Calculate the number of samples for the current category
    summary_C_df = pd.DataFrame({
        'Average Rouge-1 Score (C)': [group['rouge1_avg_skill_C'].mean()],
        'Average Rouge-2 Score (C)': [group['rouge2_avg_skill_C'].mean()],
        'Average Rouge-L Score (C)': [group['rougeL_avg_skill_C'].mean()],
        'Average Levenshtein (C)': [group['levenshtein_skill_C'].mean()],
        'Weighted Avg. Percentage Skills (C)': [group['num_skills_match_C'].sum() / group['num_skills_C'].sum()],
        'Cosine Similarity (C)': [group['average_cosine_similarity'].mean()],
        'Number of Samples': [num_samples]
    }, index=[problem_type])
    # Add the summary DataFrame for the current category to the list
    summary_C_df_list.append(summary_C_df)

# Concatenate the summary DataFrames into a single DataFrame
result_summary_df = pd.concat(summary_C_df_list)

# Calculate overall averages across all categories and add as a new row
overall_averages = {
    'Average Rouge-1 Score (C)': results_C['rouge1_avg_skill_C'].mean(),
    'Average Rouge-2 Score (C)': results_C['rouge2_avg_skill_C'].mean(),
    'Average Rouge-L Score (C)': results_C['rougeL_avg_skill_C'].mean(),
    'Average Levenshtein (C)': results_C['levenshtein_skill_C'].mean(),
    'Weighted Avg. Percentage Skills (C)': results_C['num_skills_match_C'].sum() / results_C['num_skills_C'].sum(),
    'Cosine Similarity (C)': results_C['average_cosine_similarity'].mean(),
    'Number of Samples': len(results_C)
}

overall_summary_df = pd.DataFrame(overall_averages, index=['All Samples'])

# Concatenate the overall summary as a new row to the result_summary_df
result_summary_df_C_skills = pd.concat([result_summary_df, overall_summary_df])

result_summary_df_C_skills['model'] = 'FLAN_T5_XXL'
result_summary_df_C_skills


,Average Rouge-1 Score (C),Average Rouge-2 Score (C),Average Rouge-L Score (C),Average Levenshtein (C),Weighted Avg. Percentage Skills (C),Cosine Similarity (C),Number of Samples,model
HMC,1.000000,0.500000,1.000000,0.000000,1.000000,1.000000,1,FLAN_T5_XXL
HMDC,0.377778,0.261905,0.377778,41.000000,0.400000,0.927374,10,FLAN_T5_XXL
HOWA,0.456140,0.289474,0.456140,17.210526,0.421053,0.950643,19,FLAN_T5_XXL
MC,0.786501,0.754482,0.786501,13.235294,0.753425,0.968671,34,FLAN_T5_XXL
MDC,0.515152,0.356061,0.515152,26.454545,0.521739,0.945922,11,FLAN_T5_XXL
MWA,0.597889,0.505975,0.597889,15.113208,0.267857,0.958344,53,FLAN_T5_XXL
NONE,0.666667,0.000000,0.666667,7.000000,0.666667,0.980606,3,FLAN_T5_XXL
OWA,0.619841,0.558201,0.619841,10.825397,0.515625,0.965536,63,FLAN_T5_XXL
All Samples,0.608155,0.513700,0.608155,15.435897,0.516854,0.959026,195,FLAN_T5_XXL


## Problem Types DATA RESULTS ALL SKILLS

In [35]:
# Create an empty list to store the summary DataFrames for each category
summary_ALL_df_list = []

# Group the DataFrame by "Problem Type"
grouped_results_ALL = results_ALL.groupby('Problem type')

# Iterate over each group and calculate the summary DataFrame
for problem_type, group in grouped_results_ALL:
    num_samples = len(group)  # Calculate the number of samples for the current category
    summary_ALL_df = pd.DataFrame({
        'Average Rouge-1 Score (ALL)': [group['rouge1_avg_skill_all'].mean()],
        'Average Rouge-2 Score (ALL)': [group['rouge2_avg_skill_all'].mean()],
        'Average Rouge-L Score (ALL)': [group['rougeL_avg_skill_all'].mean()],
        'Average Levenshtein (ALL)': [group['levenshtein_skill_all'].mean()],
        'Weighted Avg. Percentage Skills (ALL)': [group['num_skills_match_all'].sum() / group['num_skills_all'].sum()],
        'Cosine Similarity (ALL)': [group['average_cosine_similarity'].mean()],
        'Number of Samples': [num_samples]
    }, index=[problem_type])
    # Add the summary DataFrame for the current category to the list
    summary_ALL_df_list.append(summary_ALL_df)

# Concatenate the summary DataFrames into a single DataFrame
result_summary_ALL_df = pd.concat(summary_ALL_df_list)

# Calculate overall averages across all categories and append as a new row
overall_averages_ALL = {
    'Average Rouge-1 Score (ALL)': results_ALL['rouge1_avg_skill_all'].mean(),
    'Average Rouge-2 Score (ALL)': results_ALL['rouge2_avg_skill_all'].mean(),
    'Average Rouge-L Score (ALL)': results_ALL['rougeL_avg_skill_all'].mean(),
    'Average Levenshtein (ALL)': results_ALL['levenshtein_skill_all'].mean(),
    'Weighted Avg. Percentage Skills (ALL)': results_ALL['num_skills_match_all'].sum() / results_ALL['num_skills_all'].sum(),
    'Cosine Similarity (ALL)': results_ALL['average_cosine_similarity'].mean(),
    'Number of Samples': len(results_ALL)
}

overall_summary_ALL_df = pd.DataFrame(overall_averages_ALL, index=['All Samples'])

# Concatenate the overall summary as a new row to the result_summary_ALL_df
result_summary_df_ALL_skills = pd.concat([result_summary_ALL_df, overall_summary_ALL_df])

result_summary_df_ALL_skills['model'] = 'FLAN_T5_XXL'
result_summary_df_ALL_skills


,Average Rouge-1 Score (ALL),Average Rouge-2 Score (ALL),Average Rouge-L Score (ALL),Average Levenshtein (ALL),Weighted Avg. Percentage Skills (ALL),Cosine Similarity (ALL),Number of Samples,model
HMC,1.000000,0.333333,1.000000,0.000000,1.000000,1.000000,1,FLAN_T5_XXL
HMDC,0.920833,0.679167,0.920833,4.000000,0.862745,0.989615,10,FLAN_T5_XXL
HOWA,0.959503,0.518421,0.959503,2.789474,0.904762,0.997254,19,FLAN_T5_XXL
MC,0.974127,0.942740,0.974127,4.382353,0.932331,0.996307,34,FLAN_T5_XXL
MDC,0.915404,0.818182,0.915404,8.090909,0.893617,0.985411,11,FLAN_T5_XXL
MWA,0.970024,0.888794,0.970024,2.226415,0.910828,0.997224,53,FLAN_T5_XXL
NONE,1.000000,0.666667,1.000000,0.000000,1.000000,1.000000,3,FLAN_T5_XXL
OWA,0.971192,0.890904,0.971192,2.920635,0.913043,0.997104,63,FLAN_T5_XXL
All Samples,0.963547,0.838946,0.963547,3.271795,0.910355,0.995852,195,FLAN_T5_XXL


## Problem Types DATA RESULTS SENTENCE SKILLS

In [36]:
# Create an empty list to store the summary DataFrames for each category
summary_SENTENCE_df_list = []

# Group the DataFrame by "Problem Type"
grouped_results_SENTENCE = results_SENTENCE.groupby('Problem type')

# Iterate over each group and calculate the summary DataFrame
for problem_type, group in grouped_results_SENTENCE:
    num_samples = len(group)  # Calculate the number of samples for the current category
    summary_SENTENCE_df = pd.DataFrame({
        'Average Rouge-1 Score (SENTENCE)': [group['rouge1_score'].mean()],
        'Average Rouge-2 Score (SENTENCE)': [group['rouge2_score'].mean()],
        'Average Rouge-L Score (SENTENCE)': [group['rougeL_score'].mean()],
        'Average Levenshtein (SENTENCE)': [group['sentence_levenshtein'].mean()],
        'Cosine Similarity (SENTENCE)': [group['cosine_similarity'].mean()],
        'Number of Samples': [num_samples]
    }, index=[problem_type])
    # Add the summary DataFrame for the current category to the list
    summary_SENTENCE_df_list.append(summary_SENTENCE_df)

# Concatenate the summary DataFrames into a single DataFrame
result_summary_SENTENCE_df = pd.concat(summary_SENTENCE_df_list)

# Calculate overall averages across all categories and add as a new row
overall_averages_SENTENCE = {
    'Average Rouge-1 Score (SENTENCE)': results_SENTENCE['rouge1_score'].mean(),
    'Average Rouge-2 Score (SENTENCE)': results_SENTENCE['rouge2_score'].mean(),
    'Average Rouge-L Score (SENTENCE)': results_SENTENCE['rougeL_score'].mean(),
    'Average Levenshtein (SENTENCE)': results_SENTENCE['sentence_levenshtein'].mean(),
    'Cosine Similarity (SENTENCE)': results_SENTENCE['cosine_similarity'].mean(),
    'Number of Samples': len(results_SENTENCE)
}

overall_summary_SENTENCE_df = pd.DataFrame(overall_averages_SENTENCE, index=['All Samples'])

# Concatenate the overall summary as a new row to the result_summary_SENTENCE_df
result_summary_df_SENTENCE = pd.concat([result_summary_SENTENCE_df, overall_summary_SENTENCE_df])

result_summary_df_SENTENCE['model'] = 'FLAN_T5_XXL'
result_summary_df_SENTENCE

,Average Rouge-1 Score (SENTENCE),Average Rouge-2 Score (SENTENCE),Average Rouge-L Score (SENTENCE),Average Levenshtein (SENTENCE),Cosine Similarity (SENTENCE),Number of Samples,model
HMC,1.000000,1.000000,1.000000,1.000000,1.000000,1,FLAN_T5_XXL
HMDC,0.969111,0.914262,0.969111,10.200000,0.995074,10,FLAN_T5_XXL
HOWA,0.986129,0.957144,0.986129,3.263158,0.993202,19,FLAN_T5_XXL
MC,0.991309,0.968481,0.991309,5.676471,0.991678,34,FLAN_T5_XXL
MDC,0.976584,0.912588,0.976584,7.272727,0.977512,11,FLAN_T5_XXL
MWA,0.975634,0.949236,0.975634,3.283019,0.989095,53,FLAN_T5_XXL
NONE,1.000000,1.000000,1.000000,0.000000,1.000000,3,FLAN_T5_XXL
OWA,0.977420,0.956575,0.977420,3.857143,0.989526,63,FLAN_T5_XXL
All Samples,0.979844,0.952241,0.979844,4.415385,0.989997,195,FLAN_T5_XXL


In [37]:
result_summary_df_C_skills.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/FLAN_T5_XXL/summary_metrics_C_FLAN_T5_XXL.csv", index=False)
result_summary_df_ALL_skills.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/FLAN_T5_XXL/summary_metrics_ALL_FLAN_T5_XXL.csv", index=False)
result_summary_df_SENTENCE.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/FLAN_T5_XXL/summary_metrics_SENTENCE_FLAN_T5_XXL.csv", index=False)